In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'Country name', 'Year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect',
       'Confidence in national government', 'cont'],
      dtype='object')

In [6]:
df[df['Country name'] == "Costa Rica"]

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Country name,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,cont
419,419,419,419,419,Costa Rica,2006,7.082465,9.614259,0.936938,68.559998,0.882420,0.055333,0.797522,0.815273,0.235549,0.378860,NaN
420,420,420,420,420,Costa Rica,2007,7.432132,9.679677,0.917678,68.720001,0.922736,0.092769,0.819655,0.825872,0.240080,0.445800,NaN
421,421,421,421,421,Costa Rica,2008,6.850680,9.712777,0.915759,68.879997,0.912006,0.090521,0.815713,0.838357,0.232947,0.360781,NaN
422,422,422,422,422,Costa Rica,2009,7.614929,9.691177,0.899782,69.040001,0.886061,0.060058,0.786559,0.839989,0.217024,0.521350,NaN
423,423,423,423,423,Costa Rica,2010,7.271054,9.730945,0.915141,69.199997,0.881030,0.041907,0.762587,0.826531,0.221241,0.532539,NaN
424,424,424,424,424,Costa Rica,2011,7.228889,9.761935,0.892048,69.360001,0.926106,-0.038648,0.836583,0.793523,0.269225,0.322533,NaN
425,425,425,425,425,Costa Rica,2012,7.272250,9.797824,0.902207,69.519997,0.928914,0.040108,0.794301,0.836754,0.263027,0.280075,NaN
426,426,426,426,426,Costa Rica,2013,7.158000,9.810989,0.902069,69.680000,0.897879,0.012181,0.812863,0.808915,0.278147,0.262194,NaN
427,427,427,427,427,Costa Rica,2014,7.247086,9.834625,0.914211,69.839996,0.926707,0.003682,0.788037,0.797013,0.289529,0.404453,NaN
428,428,428,428,428,Costa Rica,2015,6.854004,9.859624,0.878273,70.000000,0.906926,-0.065053,0.761419,0.810668,0.286440,0.261169,NaN


In [7]:
df.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'Country name', 'Year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect',
       'Confidence in national government', 'cont'],
      dtype='object')

In [8]:
df.isnull().sum()

Unnamed: 0.3                           0
Unnamed: 0.2                           0
Unnamed: 0.1                           0
Unnamed: 0                             0
Country name                           0
Year                                   0
Life Ladder                            0
Log GDP per capita                     0
Social support                         0
Healthy life expectancy at birth       0
Freedom to make life choices           0
Generosity                             0
Perceptions of corruption              0
Positive affect                        0
Negative affect                        0
Confidence in national government      0
cont                                 183
dtype: int64

In [11]:
rica_df = df[df['Country name'] == 'Costa Rica']

In [12]:
rica_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 419 to 434
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0.3                       16 non-null     int64  
 1   Unnamed: 0.2                       16 non-null     int64  
 2   Unnamed: 0.1                       16 non-null     int64  
 3   Unnamed: 0                         16 non-null     int64  
 4   Country name                       16 non-null     object 
 5   Year                               16 non-null     int64  
 6   Life Ladder                        16 non-null     float64
 7   Log GDP per capita                 16 non-null     float64
 8   Social support                     16 non-null     float64
 9   Healthy life expectancy at birth   16 non-null     float64
 10  Freedom to make life choices       16 non-null     float64
 11  Generosity                         16 non-null     float64
 12

In [14]:
rica_df['cont']

419    NaN
420    NaN
421    NaN
422    NaN
423    NaN
424    NaN
425    NaN
426    NaN
427    NaN
428    NaN
429    NaN
430    NaN
431    NaN
432    NaN
433    NaN
434    NaN
Name: cont, dtype: object